In [ ]:
import copy
import random
import time

from collections import defaultdict
from IPython.display import clear_output, display, HTML


In [ ]:
# words from https://github.com/charlesreid1/five-letter-words

vocab = []
with open('sgb-words.txt', 'r') as f:
    vocab = [l.strip() for l in f]

In [ ]:
def exit(user_inp, state):
    state.is_exit_ready = True
    state.display_message = 'Goodbye, have a nice day!'
    return state


def new_game(user_inp, state):
    start = time.time()
    state.start = start
    messages = ['Game started!',
        f'Remaining guesses: {state.remaining_guesses}',
        f'Guessed words: {state.words_guessed}/{len(state.words)}'
    ]
    state.display_message = '\n'.join(messages)
    state.is_live = True
    return state

def guess_word(user_inp, state):
    if not state.is_live:
        state.display_message = 'Start game to guess!'
    else:
        if len(user_inp) != 5:
            messages = ['You must enter a 5 letter word.',
                f'Remaining guesses: {state.remaining_guesses}',
                f'Guessed words: {state.words_guessed}/{len(state.words)}'
                ]
            state.display_message = '\n'.join(messages)
            
        elif user_inp not in state.vocabulary:
            messages = [f'{user_inp} not found in the vocabulary.',
                f'Remaining guesses: {state.remaining_guesses}',
                f'Guessed words: {state.words_guessed}/{len(state.words)}'
                ]
            state.display_message = '\n'.join(messages)
        else:
            state.remaining_guesses -= 1
            messages = [f'Remaining guesses: {state.remaining_guesses}']
            state.update_board(user_inp)
            messages.append(f'Guessed words: {state.words_guessed}/{len(state.words)}')
            if state.words_guessed == len(state.words):
                state.is_exit_ready = True
                state.is_won = True
                messages.append('Congratulations! A winner is you! :D')
            elif state.remaining_guesses == 0:
                state.is_exit_ready = True
                messages.append('No guesses left! :( Better luck next time!')
            state.display_message = '\n'.join(messages)
    return state

def stats(user_inp, state):
    if state.is_live:
        state.display_message = "Finish game to display stats!"
    else:
        state.display_message = "stats TODO"

    return state


actions = {
    'e': lambda x, y: exit(x, y),
    'h': lambda x, y: help(x, y),
    'n': lambda x, y: new_game(x, y),
    's': lambda x, y: stats(x, y),
}

In [ ]:
class Word:
    def __init__(self, orientation, start, chars):
        self.orientation = orientation
        self.start = start
        self.chars = chars
        self.chars_dict = defaultdict(list)
        for i, char in enumerate(chars):
            self.chars_dict[char].append(i)
        self.is_guessed = False

    def print(self):
        print(f"Orient: {self.orientation}, start: {self.start}\n{self.chars}")
    
    def check(self, candidate):
        result_colors = ['n']*len(candidate)
        chars_dict = copy.deepcopy(self.chars_dict)
        for idx, char2 in enumerate(candidate):
            if char2 in chars_dict:
                idxs = chars_dict[char2]
                if idx in idxs:
                    result_colors[idx] = 'g'
                    idxs.remove(idx)
                    if len(idxs)==0:
                        del chars_dict[char2]
        for idx, char2 in enumerate(candidate):
            if char2 in chars_dict and result_colors[idx] != 'g':
                result_colors[idx] = 'y'
                idxs = chars_dict[char2]
                idxs.pop()
                if len(idxs)==0:
                        del chars_dict[char2]

        if len(result_colors) == len([c for c in result_colors if c=='g']):
            self.is_guessed = True
        return result_colors

In [ ]:
w = Word('h', (1,0), 'craaa')

In [ ]:
w.check('aaaia')

In [ ]:
class GameState:
    def __init__(self, vocabulary):

        self.vocabulary = vocabulary
        self.empty_board = [
            [i for i in ' ☐ ☐ '],
            [i for i in '☐'*5],
            [i for i in ' ☐ ☐ '],
            [i for i in '☐'*5],
            [i for i in ' ☐ ☐ '],
        ]
        self.empty_colors = [[None for _ in range(5)] for _ in range(5)]
        self.board = copy.deepcopy(self.empty_board)
        self.colors = copy.deepcopy(self.empty_colors)
        
        self.past_boards = []
        self.past_colors = []
        
        self.correct_board, self.words = self.generate_board()

        self.start = None
        self.stop = None

        self.is_live = False
        self.is_won = False
        self.display_message = ''
        self.remaining_guesses = 9
        self.words_guessed = 0
        self.chars_guessed = set()
        self.is_exit_ready = False
        self.start_text ='''Commands
e : exit program
h : help (how to play)
n : new game
s : statistics
'''
        self.css_str = '''
        <style>
        .yn {
            background: linear-gradient(to top right, #FFD700 49.5%, transparent 50.5%);
            }
        .ny {
            background: linear-gradient(to top right, transparent 49.5%, #FFD700 50.5%);
            }
        .gn {
            background: linear-gradient(to top right, #8FBC8F 49.5%, transparent 50.5%);
            }
        .ng {
            background: linear-gradient(to top right, transparent 49.5%, #8FBC8F 50.5%);
            }
        .yg {
            background: linear-gradient(to top right, #FFD700 49.5%, #8FBC8F 50.5%);
            }
        .gy {
            background: linear-gradient(to top right, #8FBC8F 49.5%, #FFD700 50.5%);
            }
        .g {
            background: #8FBC8F;
            }
        .y {
            background: #FFD700;
        }
        td {
            text-align: center;
            font-size: 18px;
        }  
        </style>
        '''

    def print_keyboard(self):
        row1 = 'qwertyuiop'
        row2 = 'asdfghjkl'
        row3 = 'zxcvbnm'
        print('\n'.join([' '.join(['_' if char in self.chars_guessed else char for char in row]) for row in [row1, row2, row3]]))

    def generate_test_board(self):
        #   b   i
        # c r a n e
        #   a   g
        # w i d o w
        #   n   t
        return [
            [i for i in ' b i '],
            [i for i in 'crane'],
            [i for i in ' a g '],
            [i for i in 'widow'],
            [i for i in ' n t '],
        ], [Word('h', (1,0), 'crane'), Word('h', (3,0), 'widow'), Word('v', (0,1), 'brain'), Word('v', (0,3), 'ingot')]
    
    
    def words_to_board(self, words, max_x, max_y):
        board = [[' ' for _ in range(max_x+1)] for _ in range(max_y+1)]
        for word in words:
            i,j = word.start
            for char in word.chars:
                if board[i][j] not in set([' ', char]):
                    raise ('Error checking constraints!')
                board[i][j] = char
                if word.orientation == 'h':
                    j += 1
                else:
                    i += 1
        return board

    def intersect(self, pos, orientation, words):
        constraints = []
        # print(pos, orientation)
        for word in words:
            # word.print()
            wpos, worientation = word.start, word.orientation
            if worientation == orientation:
                if worientation == 'v' and pos[1] == wpos[1] and (pos[0] <= wpos[0] <= pos[0]+4 or
                    wpos[0] <= pos[0] <= wpos[0]+4):
                    return []
                if worientation == 'h' and pos[0] == wpos[0] and (pos[1] <= wpos[1] <= pos[1]+4 or
                    wpos[1] <= pos[1] <= wpos[1]+4):
                    return []
            else:
                if orientation == 'h' and (pos[1] <= wpos[1] <= pos[1]+4) and (wpos[0] <= pos[0] <= wpos[0]+4):
                    constraints.append((wpos[1]-pos[1], word.chars[pos[0]-wpos[0]]))
                elif orientation == 'v' and (pos[0] <= wpos[0] <= pos[0]+4) and (wpos[1] <= pos[1] <= wpos[1]+4):
                    constraints.append((wpos[0]-pos[0], word.chars[pos[1]-wpos[1]]))
        # print(constraints)
        candidates = [w for w in self.vocabulary if all([w[idx]==char for idx, char in constraints])]
        
        # no duplicates
        word_chars = set([w.chars for w in words])
        candidates = [w for w in candidates if w not in word_chars]

        return candidates

    def generate_board(self, num_words=4, seed=None):
        words = []
        pos_orientations = []
        board = []
        if seed is not None:
            random.seed(seed)
        
        # #ordle pos
        fixed_pos = [((1, 0), 'h'), ((0, 1), 'v'), ((3, 0), 'h'), ((0, 3), 'v')]

        # sample word to attach to
        # sample pos to attach to
        
        
        min_x ,max_x, min_y, max_y = 0, 0, 0, 0
        while len(words) < num_words:
            # for w in words:
            #     w.print()
            # print(fixed_pos)
            # print()
            pos, orientation = fixed_pos.pop()
            candidates = self.intersect(pos, orientation, words)
            if len(candidates) == 0:
                words.pop()
                prev_pos, prev_orientation = pos_orientations.pop()
                fixed_pos.append((pos, orientation))
                fixed_pos.append((prev_pos, prev_orientation))
            else:
                new_word = Word(orientation, pos, random.choice(candidates))
                new_min_x = pos[1]
                new_max_x = pos[1]+4 if orientation == 'h' else pos[1]
                new_min_y = pos[0]
                new_max_y = pos[0]+4 if orientation == 'v' else pos[0]
                min_x = min(min_x, new_min_x)
                max_x = max(max_x, new_max_x)
                min_y = min(min_y, new_min_y)
                max_y = max(max_y, new_max_y)
                words.append(new_word)
                pos_orientations.append((pos, orientation))
        
        # shrink board s.t. min=0
        max_x -= min_x
        max_y -= min_y
        for word in words:
            word.start = (word.start[0] - min_y, word.start[1] - min_x)
        
        # print('MAX X Y: ', max_x, max_y)
        board = self.words_to_board(words, max_x, max_y)
        return board, words

    def update_board(self, candidate):
        for char in candidate:
            self.chars_guessed.add(char)

        h_words = []
        v_words = []
        for w in self.words:
            if w.orientation == 'h':
                h_words.append(w)
            else:
                v_words.append(w)
        words = h_words + v_words 
        for word in words:
            if word.is_guessed:
                colors = ['g']*5
            else:
                colors = word.check(candidate)
                if word.is_guessed:
                    self.words_guessed += 1
            for idx, color in enumerate(colors):
                if color is not None:
                    i,j = word.start
                    if word.orientation == 'h':
                        j += idx
                    else:
                        i += idx
                    if self.colors[i][j] is None:
                        self.colors[i][j] = color
                    elif self.colors[i][j] != color:
                        self.colors[i][j] += color
                    new_char = word.chars[idx] if word.is_guessed else candidate[idx]
                    if self.board[i][j] == '☐':
                        self.board[i][j] = new_char
                    elif self.board[i][j] != new_char:
                        self.board[i][j] += f'\\{new_char}'
        self.past_boards.append(self.board)
        self.past_colors.append(self.colors)
        self.board = copy.deepcopy(self.empty_board)
        self.colors = copy.deepcopy(self.empty_colors)


    def print_board(self):
        html = [self.css_str+"<table>"]
        if len(self.past_boards) == 0:
            all_boards, all_colors = [self.empty_board], [self.empty_colors]
        else:
            all_boards, all_colors = self.past_boards, self.past_colors
        for idx, (board, colors) in enumerate(zip(all_boards, all_colors)): 
            if idx%3 == 0:
                html.append('<tr>')
            html.append("<td>")    
            html.append("<table>")
            for row_b, row_c in zip(board, colors):
                html.append("<tr>") 
                for char, color in zip(row_b, row_c):
                    if color is not None:
                        html.append(f'<td class="{color}">{char.upper()}</td>')
                    else:
                        html.append(f"<td>{char.upper()}</td>")
                html.append("</tr>")
            html.append("</table>")
            html.append("</td>")
            if (idx%3 == 2) or idx == len(all_boards)-1:
                html.append('</tr>')
        return ''.join(html)

In [ ]:
state = GameState(vocabulary=vocab)
state.intersect((1,0), 'h', [Word('h', (3,0), 'widow'), Word('v', (0,1), 'brain'), Word('v', (0,3), 'ingot')])[:5]

In [ ]:
state.generate_board()

In [ ]:
state = GameState(vocabulary=vocab)

print(state.start_text)

while True:
    user_inp = input()
    clear_output()
    next_action = actions.get(user_inp, lambda x, y: guess_word(x, y))
    state = next_action(user_inp, state)
    display(HTML(state.print_board()))
    state.print_keyboard()
    if state.display_message:
        print(state.display_message)
    sys.stdout.flush()
    if state.is_exit_ready:
        state.stop = time.time()
        if state.start is not None:
            seconds = int(state.stop-state.start)
            print(f'Game took: {seconds//60}m{seconds%60}s')
            sys.stdout.flush()
        if not state.is_won:
            state.past_boards = [state.correct_board]
            state.past_colors = [state.empty_colors]
            display(HTML(state.print_board()))
        break
    